In [1]:
import os
import numpy as np
from dotenv import load_dotenv

import faiss
import google.generativeai as genai

C:\Users\PC\Code\Python\Publish-To-Git\RAG-With-LangChain-And-FAISS\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## A.　載入 .env 取得 LLM API Key

In [2]:
load_dotenv()
# print(os.getenv('GEMINI_API_KEY'))
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

# 

## B.　RAG 實作

### I.　建立向量資料庫

In [3]:
## 使用 Google Gemini 生成文字的向量嵌入
def get_gemini_embedding(text):
    response = genai.embed_content(model="models/embedding-001", content=text, task_type="retrieval_document")
    return response["embedding"]

In [4]:
documents = [
    "Google Gemini 是一款大型語言模型（LLM），由 Google DeepMind 開發，適用於各種自然語言處理（NLP）任務。",
    # "RAG（檢索增強生成）結合了檢索與 AI 生成，提升模型回答的準確度。",
    "RAG是知名品牌製作的高效清潔抹布，通常市售30TWD，基本每戶人家都會有一條RAG放置廚房做使用",
    "FAISS 是一款高效的向量搜尋庫，適合用於 AI 相關應用。",
]

In [5]:
embeddings = np.array([get_gemini_embedding(doc) for doc in documents], dtype=np.float32)
print("Embedding Shape：", embeddings.shape)
print("Content Top 5：", embeddings[:5])

Embedding Shape： (3, 768)
Content Top 5： [[ 0.04012055 -0.01553916 -0.05447042 ...  0.02586641 -0.01795555
   0.01585375]
 [ 0.06144911 -0.00659113 -0.05377533 ...  0.00689761  0.00225536
   0.00023795]
 [ 0.0564121  -0.01196416 -0.06638404 ...  0.01994789 -0.00251821
   0.0266339 ]]


In [6]:
### 建立 FAISS 索引
dimension = embeddings.shape[1]  # 嵌入向量的維度
index = faiss.IndexFlatL2(dimension) # 使用 L2 距離索引
index.add(embeddings) # 新增向量資料

### 儲存 FAISS 索引
faiss.write_index(index, "faiss_index.bin")

print("✅ FAISS 向量資料庫已建立！")

✅ FAISS 向量資料庫已建立！


### II.　使用 Gemini 進行 RAG

In [7]:
### 從 FAISS 檢索最相關的資料
def retrieve_from_faiss(query, top_k=2):
    index = faiss.read_index("faiss_index.bin") # 載入索引
    query_vector = np.array([get_gemini_embedding(query)], dtype=np.float32) # 轉換為向量
    distances, indices = index.search(query_vector, top_k) # 搜索最相關的 K 個結果
    return [documents[i] for i in indices[0]] # 返回對應的文件

In [8]:
### RAG 流程：檢索 + 生成回答
def rag_query(query):
    relevant_docs = retrieve_from_faiss(query)
    context = "\n".join(relevant_docs)

    prompt = f"""
    我希望你扮演 "基於事實回答的顧問" 的角色。
    - 請根據以下資訊回答問題 : {context}
    - 問題 : {query}
    - 遵守下列規則 : 
      * 請勿幻覺答覆
      * 繁體中文回覆
    """
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    # model = genai.GenerativeModel("gemini-pro")
    res = model.generate_content(prompt)
    return res.text

def norm_query(query):
    prompt = f"""
    我希望你扮演 "基於事實回答的顧問" 的角色。
    - 問題 : {query}
    - 遵守下列規則 : 
      * 請勿幻覺答覆
      * 繁體中文回覆
    """
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    # model = genai.GenerativeModel("gemini-pro")
    res = model.generate_content(prompt)
    return res.text

# 

## C.　成果展示

### I.　什麼是 RAG？

In [9]:
norm_query("什麼是 RAG？")

'RAG 指的是 **Retrieval Augmented Generation**，中文翻譯為 **檢索增強生成**。  它是一種技術，結合了大型語言模型 (LLM) 和外部知識庫，讓 LLM 可以在生成文本之前，先從知識庫中檢索相關資訊。  這樣可以讓 LLM 的回答更準確、更符合事實，並且避免產生幻覺（hallucination），也就是編造不存在的事實。\n\n簡單來說，RAG 系統的工作流程是：\n\n1. **接收使用者問題：** 系統接收使用者提出的問題。\n2. **檢索相關資訊：** 系統會根據問題，在預先設定好的知識庫（例如資料庫、文件集合或網頁）中搜尋相關資訊。\n3. **資訊整合：** 系統將檢索到的相關資訊整合起來。\n4. **生成答案：** 系統利用整合後的資訊，透過大型語言模型生成答案。  生成的答案會基於檢索到的事實，而非僅靠模型本身的訓練資料。\n\n因此，RAG 並非單指一種技術，而是一種架構，其關鍵在於如何有效地檢索和整合外部知識，來提升 LLM 的效能和可靠性。  不同 RAG 系統的實現方式可能有所不同，例如使用的知識庫類型、檢索演算法和 LLM 的選擇等。\n'

In [10]:
rag_query("什麼是 RAG？")

'根據提供的資訊，RAG 是一種知名品牌生產的高效清潔抹布，售價約 30 元新台幣，普遍存在於台灣家庭廚房中。  它與 AI 向量搜尋庫 FAISS 無關。\n'

### II.　什麼是 Google Gemini？

In [11]:
norm_query("什麼是 Google Gemini？")

'Google Gemini 是一個大型多模態模型（Large Multimodal Model，LMM）。它能夠處理文字、程式碼、圖片、音訊和影片等不同形式的資訊。  不同於只專注於單一模態的模型，Gemini 的多模態能力使其能更全面地理解和處理資訊，並執行更複雜的任務。\n\n目前Google公開的資訊顯示，Gemini 經過了大量的資料訓練，並展現出在多種任務上的強大能力，例如：文字生成、翻譯、問答、程式碼生成等等。  Google  將 Gemini 定位為一個基礎模型，並計劃將其應用於各種產品和服務中。  關於 Gemini 的具體技術細節，Google 尚未完全公開。  目前已知的版本包含 Gemini Pro、Gemini Ultra 和針對行動裝置優化的 Gemini Nano。  各個版本在性能和功能上存在差異。  需要注意的是，Google持續更新和改進 Gemini，其功能和能力也將隨時間推移而演進。\n'

In [12]:
rag_query("什麼是 Google Gemini？")

'Google Gemini 是一款由 Google DeepMind 開發的大型語言模型（LLM），能夠執行各種自然語言處理（NLP）任務。\n'

### III.　什麼是 ChatGPT？

In [13]:
norm_query("什麼是 ChatGPT？")

'ChatGPT 是由 OpenAI 開發的一種大型語言模型。它基於Transformer架構，並使用大量的文字數據進行訓練，使其能夠生成文字、翻譯語言、撰寫不同種類的創意內容，以及回答您的問題，即使問題的開放性和複雜性很高。  它並非一個擁有獨立意識或理解力的實體，而是通過預測文字序列中的下一個詞彙來運作，以產生看似連貫且符合語境的回應。  其能力來自於其龐大的訓練數據集，而不是真正的理解或推理能力。  需要注意的是，ChatGPT 的回覆可能包含錯誤或偏見，因為其訓練數據本身就可能存在這些問題。\n'

In [14]:
rag_query("什麼是 ChatGPT？")

'根據提供的資訊，我無法回答「什麼是ChatGPT？」這個問題。提供的資訊只描述了FAISS向量搜尋庫和一個名為RAG的清潔抹布，與ChatGPT無關。  要了解ChatGPT，需要參考其他資料。\n'

### IV.　什麼是 Agent？

In [15]:
norm_query("什麼是 Agent？")

'在電腦科學和人工智慧領域，「Agent」指的是一個能夠自主地感知環境並採取行動以達成目標的實體。  這是一個相當廣泛的概念，其具體涵蓋的範圍取決於上下文。\n\n它可以指：\n\n* **軟體代理 (Software Agent):**  這是一種常見的理解，指的是一個電腦程式，它可以獨立運作，並根據預先設定的規則或學習到的知識，在一個特定的環境中執行任務。例如，一個網路爬蟲、一個聊天機器人、一個自動交易程式，都可以被視為軟體代理。它們都具備感知（例如，從網頁獲取資訊）、推理（例如，決定哪個網頁需要訪問）、行動（例如，下載網頁內容）等能力。\n\n* **智慧型代理 (Intelligent Agent):**  這是軟體代理的一個子集，強調代理的智慧程度。它不僅能反應環境變化，還能學習、規劃、推理和解決問題，以更有效地完成任務。  例如，一個能夠根據使用者喜好推薦商品的系統，或一個能夠自動診斷故障的系統。\n\n* **多代理系統 (Multi-Agent System):**  這指的是由多個互相作用的代理組成的系統。這些代理可以合作或競爭，以達成共同目標或各自的目標。例如，一群機器人在倉庫中協同工作，或是在線上遊戲中互相競爭的玩家。\n\n\n總之，「Agent」並沒有一個單一的、絕對的定義，它是一個抽象的概念，其具體含義需要根據其所在的上下文來理解。  關鍵在於其具備自主性、感知能力和行動能力。\n'

In [16]:
rag_query("什麼是 Agent？")

'根據提供的資訊，我無法回答「什麼是Agent？」。提供的資訊僅說明FAISS是一個向量搜尋庫，以及RAG是一種清潔抹布，這兩個概念與「Agent」無關。要回答「什麼是Agent？」，需要更多關於「Agent」的背景資訊或定義。\n'

### V.　RAG 是抹布嗎? 超商可以買到它嗎?

In [17]:
norm_query("RAG 是抹布嗎? 超商可以買到它嗎?")

'RAG 不是抹布。RAG 指的是 Retrieval Augmented Generation，是一種利用外部知識庫來增強大型語言模型生成能力的技術。它是一種人工智慧概念，而非實體物品。因此，超商不可能買到它。\n'

In [18]:
rag_query("RAG 是抹布嗎? 超商可以買到它嗎?")

'根據提供的資訊，RAG是一種知名品牌生產的高效清潔抹布，市價30元台幣。  因此，答案是：\n\n* **RAG是抹布嗎？** 是的。\n* **超商可以買到它嗎？**  資訊中沒有明確指出RAG是否在超商販售，僅提到「通常市售」，因此無法確定。\n'